In [36]:
import pandas as pd
import numpy as np

### First dataset: 

In [43]:
path1 = "Data/ufc_fight_stats.csv"
stats1 = pd.read_csv(path1)
df = stats1

In [ ]:
df = df.copy()  # Ensure we are working on a fresh copy

# Standardize column names
df.columns = (df.columns.str.strip()
                        .str.replace(' ', '_')
                        .str.replace(r'[^\w]', '', regex=True)  # Remove non-alphanumeric characters
                        .str.upper())

# Function to split "X of Y" format into two columns
def split_x_of_y(column_name, df):
    if column_name in df.columns:
        # Handle missing values gracefully
        df[[f'{column_name}_MADE', f'{column_name}_ATTEMPTED']] = df[column_name].fillna('0 of 0').str.split(' of ', expand=True)
        df[f'{column_name}_MADE'] = pd.to_numeric(df[f'{column_name}_MADE'], errors='coerce').fillna(0).astype(int)
        df[f'{column_name}_ATTEMPTED'] = pd.to_numeric(df[f'{column_name}_ATTEMPTED'], errors='coerce').fillna(0).astype(int)
        df.drop(columns=[column_name], inplace=True)

# Manually define columns to split
cols_to_split = ['SIGSTR', 'TOTALSTR', 'HEAD', 'BODY', 'LEG', 'DISTANCE', 'CLINCH', 'GROUND']
for col in cols_to_split:
    split_x_of_y(col, df)

# Convert percentage columns to floats if they exist
percentage_cols = [col for col in df.columns if '%' in col]
for col in percentage_cols:
    df[col] = df[col].str.replace('%', '').astype(float) / 100

# Convert control time (CTRL) to seconds if column exists
def convert_time_to_seconds(time_str):
    if isinstance(time_str, str) and ':' in time_str:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes * 60 + seconds
    return np.nan  # Handle missing values

if 'CTRL' in df.columns:
    df['CTRL'] = df['CTRL'].apply(convert_time_to_seconds)

# Convert TD column (Takedown Attempts and Successes) if it exists
if 'TD' in df.columns:
    df[['TD_MADE', 'TD_ATTEMPTED']] = df['TD'].fillna('0 of 0').str.split(' of ', expand=True)
    df['TD_MADE'] = pd.to_numeric(df['TD_MADE'], errors='coerce').fillna(0).astype(int)
    df['TD_ATTEMPTED'] = pd.to_numeric(df['TD_ATTEMPTED'], errors='coerce').fillna(0).astype(int)
    df.drop(columns=['TD'], inplace=True)
    

# Handle missing values
df.fillna(0, inplace=True)  # Replace NaN with 0 for simplicity

Column names after standardization: ['EVENT', 'BOUT', 'ROUND', 'FIGHTER', 'KD', 'SIGSTR', 'SIGSTR_', 'TOTAL_STR', 'TD', 'TD_', 'SUBATT', 'REV', 'CTRL', 'HEAD', 'BODY', 'LEG', 'DISTANCE', 'CLINCH', 'GROUND']
Column TOTALSTR not found in DataFrame


In [39]:
df

,EVENT,BOUT,ROUND,FIGHTER,KD,SIGSTR_,TOTAL_STR,TD_,SUBATT,REV,...,LEG_MADE,LEG_ATTEMPTED,DISTANCE_MADE,DISTANCE_ATTEMPTED,CLINCH_MADE,CLINCH_ATTEMPTED,GROUND_MADE,GROUND_ATTEMPTED,TD_MADE,TD_ATTEMPTED
0,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,Round 1,Alex Pereira,0.0,47%,20 of 38,---,0.0,0.0,...,14,18,16,34,0,0,0,0,0,0
1,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,Round 2,Alex Pereira,0.0,50%,18 of 36,---,0.0,0.0,...,10,13,18,36,0,0,0,0,0,0
2,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,Round 3,Alex Pereira,0.0,61%,21 of 34,0%,0.0,0.0,...,15,15,20,33,1,1,0,0,0,1
3,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,Round 4,Alex Pereira,0.0,58%,23 of 29,---,0.0,0.0,...,1,1,1,6,6,6,0,0,0,0
4,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,Round 5,Alex Pereira,0.0,66%,15 of 22,---,0.0,0.0,...,8,8,14,21,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37697,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,Round 1,David Levicki,0.0,80%,95 of 102,---,0.0,0.0,...,0,0,1,2,2,2,1,1,0,0
37698,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Patrick Smith,0.0,100%,1 of 1,0%,1.0,0.0,...,0,0,0,0,1,1,0,0,0,1
37699,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Round 1,Ray Wizard,0.0,100%,2 of 2,---,0.0,0.0,...,1,1,1,1,0,0,0,0,0,0
37700,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,Round 1,Scott Morris,0.0,100%,2 of 2,100%,1.0,0.0,...,0,0,0,0,1,1,0,0,1,1


In [40]:
print(df.isnull().sum())  # See which columns have missing values

EVENT                 0
BOUT                  0
ROUND                 0
FIGHTER               0
KD                    0
SIGSTR_               0
TOTAL_STR             0
TD_                   0
SUBATT                0
REV                   0
CTRL                  0
SIGSTR_MADE           0
SIGSTR_ATTEMPTED      0
HEAD_MADE             0
HEAD_ATTEMPTED        0
BODY_MADE             0
BODY_ATTEMPTED        0
LEG_MADE              0
LEG_ATTEMPTED         0
DISTANCE_MADE         0
DISTANCE_ATTEMPTED    0
CLINCH_MADE           0
CLINCH_ATTEMPTED      0
GROUND_MADE           0
GROUND_ATTEMPTED      0
TD_MADE               0
TD_ATTEMPTED          0
dtype: int64


### Second data set: 

In [45]:
path2 = "Data/ufc-master.csv"
stats2 = pd.read_csv(path2)
df2 = stats2

In [46]:
# Standardize column names
df2.columns = (df2.columns.str.strip()
                        .str.replace(' ', '_')
                        .str.replace(r'[^\w]', '', regex=True)  # Remove non-alphanumeric characters
                        .str.upper())

# Handle missing values
numeric_cols = df2.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df2.select_dtypes(include=['object']).columns

# Fill missing numerical values with column mean
df2[numeric_cols] = df2[numeric_cols].apply(lambda x: x.fillna(x.mean()))

# Fill missing categorical values with 'Unknown'
df2[categorical_cols] = df2[categorical_cols].fillna('Unknown')

# Convert "FinishRoundTime" (MM:SS) to seconds
def convert_time_to_seconds(time_str):
    if isinstance(time_str, str) and ':' in time_str:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes * 60 + seconds
    return np.nan

if 'FINISHROUNDTIME' in df2.columns:
    df2['FINISHROUNDTIME_SECS'] = df2['FINISHROUNDTIME'].apply(convert_time_to_seconds)
    df2.drop(columns=['FINISHROUNDTIME'], inplace=True)

# Standardize fighter stances
stance_mapping = {
    'Orthodox': 'Orthodox',
    'orthodox': 'Orthodox',
    'Southpaw': 'Southpaw',
    'southpaw': 'Southpaw',
    'Switch': 'Switch',
    'switch': 'Switch',
    'Unknown': 'Unknown'
}
if 'REDSTANCE' in df2.columns:
    df2['REDSTANCE'] = df2['REDSTANCE'].map(stance_mapping).fillna('Unknown')
if 'BLUESTANCE' in df2.columns:
    df2['BLUESTANCE'] = df2['BLUESTANCE'].map(stance_mapping).fillna('Unknown')

# Normalize Red vs Blue Differences
if all(col in df2.columns for col in ['REDWINS', 'BLUEWINS']):
    df2['WINDIF'] = df2['REDWINS'] - df2['BLUEWINS']

if all(col in df2.columns for col in ['REDREACHCMS', 'BLUEREACHCMS', 'REDHEIGHTCMS', 'BLUEHEIGHTCMS', 'REDAGE', 'BLUEAGE']):
    df2['REACHDIF'] = df2['REDREACHCMS'] - df2['BLUEREACHCMS']
    df2['HEIGHTDIF'] = df2['REDHEIGHTCMS'] - df2['BLUEHEIGHTCMS']
    df2['AGEDIF'] = df2['REDAGE'] - df2['BLUEAGE']

C:\Users\emilp\AppData\Local\Temp\ipykernel_39712\4136878062.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2['FINISHROUNDTIME_SECS'] = df2['FINISHROUNDTIME'].apply(convert_time_to_seconds)


In [40]:
print(df2.isnull().sum())  # See which columns have missing values

REDFIGHTER                0
BLUEFIGHTER               0
REDODDS                   0
BLUEODDS                  0
REDEXPECTEDVALUE          0
                       ... 
RSUBODDS                  0
BSUBODDS                  0
RKOODDS                   0
BKOODDS                   0
FINISHROUNDTIME_SECS    622
Length: 118, dtype: int64


In [44]:
print("DF columns:", df.columns.tolist())
print("DF2 columns:", df2.columns.tolist())

DF columns: ['EVENT', 'BOUT', 'ROUND', 'FIGHTER', 'KD', 'SIGSTR_', 'TOTAL_STR', 'TD_', 'SUBATT', 'REV', 'CTRL', 'SIGSTR_MADE', 'SIGSTR_ATTEMPTED', 'HEAD_MADE', 'HEAD_ATTEMPTED', 'BODY_MADE', 'BODY_ATTEMPTED', 'LEG_MADE', 'LEG_ATTEMPTED', 'DISTANCE_MADE', 'DISTANCE_ATTEMPTED', 'CLINCH_MADE', 'CLINCH_ATTEMPTED', 'GROUND_MADE', 'GROUND_ATTEMPTED', 'TD_MADE', 'TD_ATTEMPTED']
DF2 columns: ['REDFIGHTER', 'BLUEFIGHTER', 'REDODDS', 'BLUEODDS', 'REDEXPECTEDVALUE', 'BLUEEXPECTEDVALUE', 'DATE', 'LOCATION', 'COUNTRY', 'WINNER', 'TITLEBOUT', 'WEIGHTCLASS', 'GENDER', 'NUMBEROFROUNDS', 'BLUECURRENTLOSESTREAK', 'BLUECURRENTWINSTREAK', 'BLUEDRAWS', 'BLUEAVGSIGSTRLANDED', 'BLUEAVGSIGSTRPCT', 'BLUEAVGSUBATT', 'BLUEAVGTDLANDED', 'BLUEAVGTDPCT', 'BLUELONGESTWINSTREAK', 'BLUELOSSES', 'BLUETOTALROUNDSFOUGHT', 'BLUETOTALTITLEBOUTS', 'BLUEWINSBYDECISIONMAJORITY', 'BLUEWINSBYDECISIONSPLIT', 'BLUEWINSBYDECISIONUNANIMOUS', 'BLUEWINSBYKO', 'BLUEWINSBYSUBMISSION', 'BLUEWINSBYTKODOCTORSTOPPAGE', 'BLUEWINS', 'BLUESTA

In [ ]:
import pandas as pd
import numpy as np

# Ensure lowercase and trimmed fighter names
df['FIGHTER'] = df['FIGHTER'].str.strip().str.lower()
df2['REDFIGHTER'] = df2['REDFIGHTER'].str.strip().str.lower()
df2['BLUEFIGHTER'] = df2['BLUEFIGHTER'].str.strip().str.lower()

# Merge on REDFIGHTER
merged_red = df.merge(df2, left_on='FIGHTER', right_on='REDFIGHTER', how='left', suffixes=('', '_RED'))

# Merge on BLUEFIGHTER
merged_blue = df.merge(df2, left_on='FIGHTER', right_on='BLUEFIGHTER', how='left', suffixes=('', '_BLUE'))

# Combine both merged DataFrames
df_combined = pd.concat([merged_red, merged_blue], ignore_index=True)

# Remove duplicated columns, if any
df_combined = df_combined.loc[:, ~df_combined.columns.duplicated()]

# Fill any remaining missing values
df_combined.fillna(0, inplace=True)

# Extract the number from 'Round X' safely, keeping NaNs intact
df_combined['ROUND'] = df_combined['ROUND'].str.extract(r'(\d+)')  # Extract digits
df_combined['ROUND'] = pd.to_numeric(df_combined['ROUND'], errors='coerce')  # Convert to numeric

# Fill NaNs with 0 or any default value you want, then convert to int
df_combined['ROUND'] = df_combined['ROUND'].fillna(0).astype(int)

# Save or preview
df_combined.to_csv("Data/combined_ufc_dataset_2.csv", index=False)
print("Combined dataset shape:", df_combined.shape)
print(df_combined.head())

Combined dataset shape: (392717, 30)
                           EVENT                               BOUT  ROUND  \
0  UFC 313: Pereira vs. Ankalaev  Alex Pereira vs. Magomed Ankalaev      1   
1  UFC 313: Pereira vs. Ankalaev  Alex Pereira vs. Magomed Ankalaev      1   
2  UFC 313: Pereira vs. Ankalaev  Alex Pereira vs. Magomed Ankalaev      1   
3  UFC 313: Pereira vs. Ankalaev  Alex Pereira vs. Magomed Ankalaev      1   
4  UFC 313: Pereira vs. Ankalaev  Alex Pereira vs. Magomed Ankalaev      1   

        FIGHTER   KD SIGSTR_ TOTAL_STR  TD_  SUBATT  REV  ...  \
0  alex pereira  0.0     47%  20 of 38  ---     0.0  0.0  ...   
1  alex pereira  0.0     47%  20 of 38  ---     0.0  0.0  ...   
2  alex pereira  0.0     47%  20 of 38  ---     0.0  0.0  ...   
3  alex pereira  0.0     47%  20 of 38  ---     0.0  0.0  ...   
4  alex pereira  0.0     47%  20 of 38  ---     0.0  0.0  ...   

   DISTANCE_ATTEMPTED  CLINCH_MADE  CLINCH_ATTEMPTED  GROUND_MADE  \
0                  34            0

In [48]:
df_combined

,EVENT,BOUT,ROUND,FIGHTER,KD,SIGSTR_,TOTAL_STR,TD_,SUBATT,REV,...,DISTANCE_ATTEMPTED,CLINCH_MADE,CLINCH_ATTEMPTED,GROUND_MADE,GROUND_ATTEMPTED,TD_MADE,TD_ATTEMPTED,REDFIGHTER,DATE,BLUEFIGHTER
0,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,1,alex pereira,0.0,47%,20 of 38,---,0.0,0.0,...,34,0,0,0,0,0,0,alex pereira,05/10/2024,0
1,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,1,alex pereira,0.0,47%,20 of 38,---,0.0,0.0,...,34,0,0,0,0,0,0,alex pereira,29/06/2024,0
2,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,1,alex pereira,0.0,47%,20 of 38,---,0.0,0.0,...,34,0,0,0,0,0,0,alex pereira,13/04/2024,0
3,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,1,alex pereira,0.0,47%,20 of 38,---,0.0,0.0,...,34,0,0,0,0,0,0,alex pereira,08/04/2023,0
4,UFC 313: Pereira vs. Ankalaev,Alex Pereira vs. Magomed Ankalaev,1,alex pereira,0.0,47%,20 of 38,---,0.0,0.0,...,34,0,0,0,0,0,0,alex pereira,12/03/2022,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392712,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,1,david levicki,0.0,80%,95 of 102,---,0.0,0.0,...,2,2,2,1,1,0,0,0,0,0
392713,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,1,patrick smith,0.0,100%,1 of 1,0%,1.0,0.0,...,0,1,1,0,0,0,1,0,0,0
392714,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,1,ray wizard,0.0,100%,2 of 2,---,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
392715,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,1,scott morris,0.0,100%,2 of 2,100%,1.0,0.0,...,0,1,1,0,0,1,1,0,0,0


In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load dataset
df = pd.read_csv("Data/ufc-master.csv")  # Adjust path if needed

# Step 1: Normalize fighter names (correct case capitalization)
df['RedFighter'] = df['RedFighter'].str.strip().str.title()  # Normalize Red fighter names
df['BlueFighter'] = df['BlueFighter'].str.strip().str.title()  # Normalize Blue fighter names

# Step 2: Replace 'Red' and 'Blue' columns with the actual fighter names
df['Fighter'] = df['RedFighter']
df['Opponent'] = df['BlueFighter']

# Drop the original 'RedFighter' and 'BlueFighter' columns after replacing them
df = df.drop(columns=['RedFighter', 'BlueFighter'])

# Step 3: Create a new 'Age' column based on the max age between 'RedAge' and 'BlueAge'
df['Age'] = df[['RedAge', 'BlueAge']].max(axis=1)

# Step 4: Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Step 5: Find the latest fight date for each fighter (whether Red or Blue)
# We will create two separate date columns: one for RedFighter and one for BlueFighter
df['RedDate'] = df['Date']
df['BlueDate'] = df['Date']

# Now we need to find the latest fight date for each fighter
latest_dates = pd.concat([df[['Fighter', 'RedDate']], df[['Fighter', 'BlueDate']]], axis=0)

# Drop duplicates to make sure the fighter appears once with their latest fight date
latest_dates = latest_dates.groupby('Fighter').max().reset_index()

# Merge this latest date information back into the fighter_stats DataFrame
df = df.merge(latest_dates, on='Fighter', how='left')

# Step 6: List of columns to aggregate (including both numeric and categorical columns)
columns_to_aggregate = [
    'RedOdds', 'BlueOdds', 'RedExpectedValue', 'BlueExpectedValue', 'Winner', 'TitleBout', 'WeightClass', 
    'Gender', 'NumberOfRounds', 'BlueCurrentLoseStreak', 'BlueCurrentWinStreak', 'BlueDraws', 
    'BlueAvgSigStrLanded', 'BlueAvgSigStrPct', 'BlueAvgSubAtt', 'BlueAvgTDLanded', 'BlueAvgTDPct', 
    'BlueLongestWinStreak', 'BlueLosses', 'BlueTotalRoundsFought', 'BlueTotalTitleBouts', 
    'BlueWinsByDecisionMajority', 'BlueWinsByDecisionSplit', 'BlueWinsByDecisionUnanimous', 
    'BlueWinsByKO', 'BlueWinsBySubmission', 'BlueWinsByTKODoctorStoppage', 'BlueWins', 'BlueStance', 
    'BlueHeightCms', 'BlueReachCms', 'BlueWeightLbs', 'RedCurrentLoseStreak', 'RedCurrentWinStreak', 
    'RedDraws', 'RedAvgSigStrLanded', 'RedAvgSigStrPct', 'RedAvgSubAtt', 'RedAvgTDLanded', 'RedAvgTDPct', 
    'RedLongestWinStreak', 'RedLosses', 'RedTotalRoundsFought', 'RedTotalTitleBouts', 
    'RedWinsByDecisionMajority', 'RedWinsByDecisionSplit', 'RedWinsByDecisionUnanimous', 'RedWinsByKO', 
    'RedWinsBySubmission', 'RedWinsByTKODoctorStoppage', 'RedWins', 'RedStance', 'RedHeightCms', 
    'RedReachCms', 'RedWeightLbs', 'LoseStreakDif', 'WinStreakDif', 'LongestWinStreakDif', 
    'WinDif', 'LossDif', 'TotalRoundDif', 'TotalTitleBoutDif', 'KODif', 'SubDif', 'HeightDif', 'ReachDif', 
    'AgeDif', 'SigStrDif', 'AvgSubAttDif', 'AvgTDDif', 'EmptyArena', 'BMatchWCRank', 'RMatchWCRank', 
    'RWFlyweightRank', 'RWFeatherweightRank', 'RWStrawweightRank', 'RWBantamweightRank', 'RHeavyweightRank', 
    'RLightHeavyweightRank', 'RMiddleweightRank', 'RWelterweightRank', 'RLightweightRank', 'RFeatherweightRank', 
    'RBantamweightRank', 'RFlyweightRank', 'RPFPRank', 'BWFlyweightRank', 'BWFeatherweightRank', 'BWStrawweightRank', 
    'BWBantamweightRank', 'BHeavyweightRank', 'BLightHeavyweightRank', 'BMiddleweightRank', 'BWelterweightRank', 
    'BLightweightRank', 'BFeatherweightRank', 'BBantamweightRank', 'BFlyweightRank', 'BPFPRank', 'BetterRank', 
    'Finish', 'FinishDetails', 'FinishRound', 'FinishRoundTime', 'TotalFightTimeSecs', 'RedDecOdds', 'BlueDecOdds', 
    'RSubOdds', 'BSubOdds', 'RKOOdds', 'BKOOdds', 'Age'  # Ensure 'Age' is included
]

# Step 7: Filter columns to only numeric columns for aggregation
numeric_columns = df[columns_to_aggregate].select_dtypes(include=[np.number]).columns.tolist()

# Step 8: Ensure all columns to aggregate are numeric
# Convert any non-numeric columns (if any) to numeric, forcing errors to NaN (so they get excluded)
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Step 9: Aggregate by averaging the statistics for each fighter
fighter_stats = df.groupby('Fighter')[numeric_columns].mean().reset_index()

# Step 10: Normalize the features (only numeric columns except 'Age')
scaler = MinMaxScaler()

# Remove 'Age' from normalization
numeric_columns_without_age = [col for col in numeric_columns if col != 'Age']

# Normalize the stats (for all numeric columns except 'Age')
fighter_stats[numeric_columns_without_age] = scaler.fit_transform(fighter_stats[numeric_columns_without_age])

# Step 11: Add the latest date for each fighter
fighter_stats = fighter_stats.merge(latest_dates[['Fighter', 'RedDate']], on='Fighter', how='left')

# Step 12: Save final result to CSV
fighter_stats.to_csv("Data/fighter_fpi_normalized.csv", index=False)

# Step 13: Display top fighters
print(fighter_stats)


                 Fighter   RedOdds  BlueOdds  RedExpectedValue  \
0             Aalon Cruz  0.664348  0.593478          0.062149   
1         Aaron Phillips  0.783478  0.446739          0.191808   
2            Aaron Riley  0.733913  0.506522          0.169088   
3             Aaron Rosa  0.688696  0.569565          0.102009   
4          Aaron Simpson  0.706087  0.543043          0.106848   
...                  ...       ...       ...               ...   
1653        Zhang Lipeng  0.791884  0.397101          0.310584   
1654      Zhang Mingyang  0.672174  0.543478          0.081682   
1655         Zhang Weili  0.632609  0.620652          0.047936   
1656    Zubaira Tukhugov  0.623478  0.638043          0.037071   
1657  Zygimantas Ramaska  0.772174  0.460000          0.149614   

      BlueExpectedValue  NumberOfRounds  BlueCurrentLoseStreak  \
0              0.143511            0.00               0.000000   
1              0.046022            0.00               0.125000   
2        

In [7]:
# Check column names of the dataset to ensure the correct names are used
print(df.columns)

Index(['RedFighter', 'BlueFighter', 'RedOdds', 'BlueOdds', 'RedExpectedValue',
       'BlueExpectedValue', 'Date', 'Location', 'Country', 'Winner',
       ...
       'FinishDetails', 'FinishRound', 'FinishRoundTime', 'TotalFightTimeSecs',
       'RedDecOdds', 'BlueDecOdds', 'RSubOdds', 'BSubOdds', 'RKOOdds',
       'BKOOdds'],
      dtype='object', length=118)


In [55]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load dataset
df = pd.read_csv("Data/ufc-master.csv")  # Adjust path if needed

# Step 1: Normalize fighter names (correct case capitalization)
df['RedFighter'] = df['RedFighter'].str.strip().str.title()  # Normalize Red fighter names
df['BlueFighter'] = df['BlueFighter'].str.strip().str.title()  # Normalize Blue fighter names

# Step 2: Replace 'Red' and 'Blue' columns with the actual fighter names
df['Fighter'] = df['RedFighter']
df['Opponent'] = df['BlueFighter']

# Drop the original 'RedFighter' and 'BlueFighter' columns after replacing them
df = df.drop(columns=['RedFighter', 'BlueFighter'])

# Step 3: Create a new 'Age' column based on the max age between 'RedAge' and 'BlueAge'
df['Age'] = df[['RedAge', 'BlueAge']].max(axis=1)

# Step 4: Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Step 5: Find the latest fight date for each fighter (whether Red or Blue)
# We will create two separate date columns: one for RedFighter and one for BlueFighter
df['RedDate'] = df['Date']
df['BlueDate'] = df['Date']

# Now we need to find the latest fight date for each fighter
latest_dates = pd.concat([df[['Fighter', 'RedDate']], df[['Fighter', 'BlueDate']]], axis=0)

# Drop duplicates to make sure the fighter appears once with their latest fight date
latest_dates = latest_dates.groupby('Fighter').max().reset_index()

# Merge this latest date information back into the fighter_stats DataFrame
df = df.merge(latest_dates, on='Fighter', how='left')

# Step 6: Create the 'Wins' column (max of RedWins and BlueWins)
df['Wins'] = df[['RedWins', 'BlueWins']].max(axis=1)

# Step 7: List of columns to aggregate (including both numeric and categorical columns)
columns_to_aggregate = [
    'RedOdds', 'BlueOdds', 'RedExpectedValue', 'BlueExpectedValue', 'Winner', 'TitleBout', 'WeightClass', 
    'Gender', 'NumberOfRounds', 'BlueCurrentLoseStreak', 'BlueCurrentWinStreak', 'BlueDraws', 
    'BlueAvgSigStrLanded', 'BlueAvgSigStrPct', 'BlueAvgSubAtt', 'BlueAvgTDLanded', 'BlueAvgTDPct', 
    'BlueLongestWinStreak', 'BlueLosses', 'BlueTotalRoundsFought', 'BlueTotalTitleBouts', 
    'BlueWinsByDecisionMajority', 'BlueWinsByDecisionSplit', 'BlueWinsByDecisionUnanimous', 
    'BlueWinsByKO', 'BlueWinsBySubmission', 'BlueWinsByTKODoctorStoppage', 'BlueWins', 'BlueStance', 
    'BlueHeightCms', 'BlueReachCms', 'BlueWeightLbs', 'RedCurrentLoseStreak', 'RedCurrentWinStreak', 
    'RedDraws', 'RedAvgSigStrLanded', 'RedAvgSigStrPct', 'RedAvgSubAtt', 'RedAvgTDLanded', 'RedAvgTDPct', 
    'RedLongestWinStreak', 'RedLosses', 'RedTotalRoundsFought', 'RedTotalTitleBouts', 
    'RedWinsByDecisionMajority', 'RedWinsByDecisionSplit', 'RedWinsByDecisionUnanimous', 'RedWinsByKO', 
    'RedWinsBySubmission', 'RedWinsByTKODoctorStoppage', 'RedWins', 'RedStance', 'RedHeightCms', 
    'RedReachCms', 'RedWeightLbs', 'LoseStreakDif', 'WinStreakDif', 'LongestWinStreakDif', 
    'WinDif', 'LossDif', 'TotalRoundDif', 'TotalTitleBoutDif', 'KODif', 'SubDif', 'HeightDif', 'ReachDif', 
    'AgeDif', 'SigStrDif', 'AvgSubAttDif', 'AvgTDDif', 'EmptyArena', 'BMatchWCRank', 'RMatchWCRank', 
    'RWFlyweightRank', 'RWFeatherweightRank', 'RWStrawweightRank', 'RWBantamweightRank', 'RHeavyweightRank', 
    'RLightHeavyweightRank', 'RMiddleweightRank', 'RWelterweightRank', 'RLightweightRank', 'RFeatherweightRank', 
    'RBantamweightRank', 'RFlyweightRank', 'RPFPRank', 'BWFlyweightRank', 'BWFeatherweightRank', 'BWStrawweightRank', 
    'BWBantamweightRank', 'BHeavyweightRank', 'BLightHeavyweightRank', 'BMiddleweightRank', 'BWelterweightRank', 
    'BLightweightRank', 'BFeatherweightRank', 'BBantamweightRank', 'BFlyweightRank', 'BPFPRank', 'BetterRank', 
    'Finish', 'FinishDetails', 'FinishRound', 'FinishRoundTime', 'TotalFightTimeSecs', 'RedDecOdds', 'BlueDecOdds', 
    'RSubOdds', 'BSubOdds', 'RKOOdds', 'BKOOdds', 'Age'  # Ensure 'Age' is included
]

# Step 8: Filter columns to only numeric columns for aggregation
numeric_columns = df[columns_to_aggregate].select_dtypes(include=[np.number]).columns.tolist()

# Step 9: Ensure all columns to aggregate are numeric
# Convert any non-numeric columns (if any) to numeric, forcing errors to NaN (so they get excluded)
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Step 10: Aggregate by averaging the statistics for each fighter
fighter_stats = df.groupby('Fighter')[numeric_columns].mean().reset_index()

# Step 11: Normalize the features (only numeric columns except 'Age' and 'Wins')
scaler = MinMaxScaler()

# Remove 'Age' and 'Wins' from normalization
numeric_columns_without_age_wins = [col for col in numeric_columns if col not in ['Age', 'Wins']]

# Normalize the stats (for all numeric columns except 'Age' and 'Wins')
fighter_stats[numeric_columns_without_age_wins] = scaler.fit_transform(fighter_stats[numeric_columns_without_age_wins])

# Step 12: Add the latest date for each fighter
fighter_stats = fighter_stats.merge(latest_dates[['Fighter', 'RedDate']], on='Fighter', how='left')

# Step 13: Save final result to CSV
fighter_stats.to_csv("Data/fighter_fpi_normalized_2.csv", index=False)

# Step 14: Display top fighters
print(fighter_stats)


                 Fighter   RedOdds  BlueOdds  RedExpectedValue  \
0             Aalon Cruz  0.664348  0.593478          0.062149   
1         Aaron Phillips  0.783478  0.446739          0.191808   
2            Aaron Riley  0.733913  0.506522          0.169088   
3             Aaron Rosa  0.688696  0.569565          0.102009   
4          Aaron Simpson  0.706087  0.543043          0.106848   
...                  ...       ...       ...               ...   
1653        Zhang Lipeng  0.791884  0.397101          0.310584   
1654      Zhang Mingyang  0.672174  0.543478          0.081682   
1655         Zhang Weili  0.632609  0.620652          0.047936   
1656    Zubaira Tukhugov  0.623478  0.638043          0.037071   
1657  Zygimantas Ramaska  0.772174  0.460000          0.149614   

      BlueExpectedValue  NumberOfRounds  BlueCurrentLoseStreak  \
0              0.143511            0.00               0.000000   
1              0.046022            0.00               0.125000   
2        

In [56]:
import pandas as pd

# Load the datasets
fpi_df = pd.read_csv("Data/fighter_fpi_normalized.csv")  # FPI dataset
additional_fighters_df = pd.read_csv("Data/ufc-fighters-statistics.csv")  # Dataset with more fighters

# Ensure that fighter names are consistently formatted (strip whitespace, proper case)
fpi_df['Fighter'] = fpi_df['Fighter'].str.strip().str.title()
additional_fighters_df['Fighter'] = additional_fighters_df['Fighter'].str.strip().str.title()

# Merge the datasets on the 'Fighter' column to get wins, losses, and draws
# This keeps only fighters that exist in the fpi_df and includes their wins, losses, and draws from additional_fighters_df
merged_df = pd.merge(fpi_df, additional_fighters_df[['Fighter', 'Wins', 'Losses', 'Draws']], 
                     on='Fighter', how='inner')

# Step 4: Optionally, you can reorder the columns or save the merged dataset
merged_df.to_csv("Data/final_fighter_dataset.csv", index=False)

# Display the merged dataframe (top 10 rows as an example)
print(merged_df.head())


          Fighter   RedOdds  BlueOdds  RedExpectedValue  BlueExpectedValue  \
0      Aalon Cruz  0.664348  0.593478          0.062149           0.143511   
1  Aaron Phillips  0.783478  0.446739          0.191808           0.046022   
2     Aaron Riley  0.733913  0.506522          0.169088           0.089313   
3      Aaron Rosa  0.688696  0.569565          0.102009           0.093130   
4   Aaron Simpson  0.706087  0.543043          0.106848           0.152744   

   NumberOfRounds  BlueCurrentLoseStreak  BlueCurrentWinStreak  BlueDraws  \
0             0.0                  0.000              0.000000        0.0   
1             0.0                  0.125              0.000000        0.0   
2             0.0                  0.250              0.000000        0.0   
3             0.0                  0.000              0.000000        0.0   
4             0.0                  0.200              0.066667        0.0   

   BlueAvgSigStrLanded  ...  BlueDecOdds  RSubOdds  BSubOdds   RKOOd